In [3]:
#load libraries 

#required libraries for detectron2
#PyTorch ≥ 1.5 and torchvision that matches the PyTorch installation. You can install them together at pytorch.org to make sure of this
#!pip install torchvision
#OpenCV is optional and needed by demo and visualization
#!pip install opencv-python
#This will only work on Linux or macOS
####if Detectron2 is missing:
#!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


import os
import numpy as np
import json
import random
import cv2
from detectron2 import model_zoo
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode

In [9]:
#set here your variables

#train: Annotated Images for training
#train.json: Annotations + Path to Images for all images stored in train
#test: Annotated Images for model evaluation (Taken from train)
#test.json: Like train.json but for test
#test_set2: Unannotated images which do not stem from the train data to evaluate generalization of the model

#please ensure the following structure:
# |
# |-working directory
#     |-train
#     |   |-img1.jpg
#     |   |-img2.jpg
#     |   |-[...]
#     |   |-train.json
#     |
#     |-test
#     |   |-img1.jpg
#     |   |-img2.jpg
#     |   |-[...]
#     |   |-test.json
#     |
#     |-test_set2
#         |-img1.jpg
#         |-img2.jpg
#         |-[...]

#working directory 
working_dir="/home/leto/Training_C_AI_DATA/k_means"
#Save Model Weights to
output_dir_model="/home/leto/Training_C_AI_DATA/k_means/1000"
#save pictures with prediction
output_dir_imgs="/home/leto/Training_C_AI_DATA/k_means/1000"
############################################
####OPTIMIZE HERE FOR BETTER PERFORMANCE####
#Iterations for training
training_iterations=1000
#Number of workers
worker_num=2
#Images per Batch (a higher number will need more VRAM, but is also good for stability in the learning process)
batchsize=2
#Base learning rate
base_lr=0.00025
#Number of classes (always +1 bc of background)
num_classes=10
#detection threshold (threshold for a class probability upon which a prediction will be considered to be true)
detect_thresh=0.55
#if test_set2 was renamed
test_set2="test_set2"


#####DO NOT CHANGE#####
train_dir=os.path.join(working_dir,"train")
train_json="train.json"

In [10]:
json_train_file = os.path.join(working_dir, "train", train_json)
#load dataset and create coco dataset for DETECTRON2
with open(json_train_file) as f:
    imgs_anns = json.load(f)

register_coco_instances("train", {}, os.path.join(working_dir,"train","train.json"), "train")
register_coco_instances("test", {}, os.path.join(working_dir,"test","test.json"), "test")

dataset_dicts = DatasetCatalog.get("train")
dataset_metadata = MetadataCatalog.get("train")

#a warning about the category ids will appear - this is okay

AssertionError: Dataset 'train' is already registered!

In [11]:
#create the config file for training
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = worker_num
cfg.OUTPUT_DIR = output_dir_model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = batchsize
cfg.SOLVER.BASE_LR = base_lr # pick a good LR
cfg.SOLVER.MAX_ITER = training_iterations
cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
cfg.nms = True

In [12]:
#THIS WILL START THE TRAINER, which can be time consuming
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[11/30 15:46:51 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.


[11/30 15:46:51 d2.engine.train_loop]: Starting training from iteration 0
[11/30 15:47:01 d2.utils.events]:  eta: 0:08:28  iter: 19  total_loss: 3.404  loss_cls: 2.647  loss_box_reg: 0.66  loss_rpn_cls: 0.04355  loss_rpn_loc: 0.04445  time: 0.5098  data_time: 0.3318  lr: 4.9953e-06  max_mem: 4394M
[11/30 15:47:12 d2.utils.events]:  eta: 0:08:21  iter: 39  total_loss: 3.343  loss_cls: 2.501  loss_box_reg: 0.7033  loss_rpn_cls: 0.08192  loss_rpn_loc: 0.0477  time: 0.5171  data_time: 0.3405  lr: 9.9902e-06  max_mem: 4394M
[11/30 15:47:22 d2.utils.events]:  eta: 0:08:11  iter: 59  total_loss: 2.986  loss_cls: 2.248  loss_box_reg: 0.6379  loss_rpn_cls: 0.03773  loss_rpn_loc: 0.04215  time: 0.5212  data_time: 0.3468  lr: 1.4985e-05  max_mem: 4394M
[11/30 15:47:33 d2.utils.events]:  eta: 0:08:00  iter: 79  total_loss: 2.541  loss_cls: 1.88  loss_box_reg: 0.5524  loss_rpn_cls: 0.01858  loss_rpn_loc: 0.02505  time: 0.5235  data_time: 0.3465  lr: 1.998e-05  max_mem: 4394M
[11/30 15:47:43 d2.util

In [13]:
# Look at training curves in tensorboard:
!kill 16886
%load_ext tensorboard
%tensorboard --logdir output

/bin/bash: line 0: kill: (16886) - No such process


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 81380.

In [14]:
#config for test mode
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = detect_thresh # set the testing threshold for this model
cfg.DATASETS.TEST = ("test", )
predictor = DefaultPredictor(cfg)

In [15]:
#prepare test dataset
dataset_dicts_test = DatasetCatalog.get("test")
dataset_metadata_test = MetadataCatalog.get("test")


WARNING [11/30 15:59:29 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/30 15:59:29 d2.data.datasets.coco]: Loaded 48 images in COCO format from /data/collembola_ai/test/test.json


In [18]:
#annotate all pictures of testset1 and testset2 with the predictions of the trained model
os.makedirs(output_dir_imgs, exist_ok=True)
output_testset1 = os.path.join(output_dir_imgs, "testset1")
os.makedirs(output_testset1, exist_ok=True)
output_testset2 = os.path.join(output_dir_imgs, "testset2")
os.makedirs(output_testset2, exist_ok=True)

i = 0
for d in dataset_dicts_test:
    #create variable with output name
    output_name = output_testset1 + "/annotated_" + str(i) + ".jpg"
    #load image
    img = cv2.imread(d["file_name"])
    #make prediction
    outputs = predictor(img)
    #draw prediction
    visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_metadata_test, scale=1.)
    instances = outputs["instances"].to("cpu")
    vis = visualizer.draw_instance_predictions(instances)
    result = vis.get_image()[:, :, ::-1]
    #write image
    write_res = cv2.imwrite(output_name, result)
    i += 1
    break

for i in os.listdir(os.path.join(working_dir,test_set2)):
    if ".jpg" in i:
        file_path = os.path.join(working_dir, test_set2, i)
        im = cv2.imread(file_path)
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1], metadata=dataset_metadata, scale=1.)
        instances = outputs["instances"].to("cpu")
        v = v.draw_instance_predictions(instances)
        result = v.get_image()[:, :, ::-1]
        output_name = output_testset2 + "/annotated_" + str(i) + ".jpg"
        write_res = cv2.imwrite(output_name, result)
        break

In [19]:
#Input /path/to/input
input_single_img="/data/collembola_ai/mix_plate4.jpg"
#Output (just name, will be saved in $output_dir_imgs/single_images)
output_single_name="test.jpg"

#single image prediction
#define outputname / path
single_img_output=os.path.join(output_dir_imgs,"single_images")
os.makedirs(single_img_output, exist_ok=True)

output_name = os.path.join(single_img_output,output_single_name)
#load an image
img = cv2.imread(input_single_img)
outputs = predictor(img)
visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_metadata_test, scale=1.)
instances = outputs["instances"].to("cpu")
vis = visualizer.draw_instance_predictions(instances)
result = vis.get_image()[:, :, ::-1]
write_res = cv2.imwrite(output_name, result)

In [20]:
#IN-BUILD MODEL EVALUATION
#I did not found this really useful since the evaluation takes the accurate position strongly into account, something what we do not care as much about...
evaluator = COCOEvaluator("test", cfg, False, output_dir=output_dir_imgs)
val_loader = build_detection_test_loader(cfg, "test")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [11/30 16:06:24 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [11/30 16:06:24 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/30 16:06:24 d2.data.datasets.coco]: Loaded 48 images in COCO format from /data/collembola_ai/test/test.json
[11/30 16:06:24 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |   category   | #instances   |   category    | #instances   |
|:-------------:|:-------------|:------------:|:-------------|:-------------:|:-------------|
| ceratophyse.. | 9            |   desoria    | 36           | deuterosmin.. | 58           |
|   folsomia    | 55           | megalothorax | 23           | pseudosinella | 25           |
|    sinella    | 17           | sminthurides | 52           |   species01   | 58           |
|  sphaeridia   | 52       

OrderedDict([('bbox',
              {'AP': 48.89002297980867,
               'AP50': 76.94115935671701,
               'AP75': 54.2543081817539,
               'APs': nan,
               'APm': 65.0,
               'APl': 48.81011232758113,
               'AP-ceratophysella': 42.113586358635864,
               'AP-desoria': 57.813182631438664,
               'AP-deuterosminthurus': 63.19432856586978,
               'AP-folsomia': 34.02350778229443,
               'AP-megalothorax': 34.64653335450399,
               'AP-pseudosinella': 80.43384832988794,
               'AP-sinella': 48.52014998251747,
               'AP-sminthurides': 50.5421676086563,
               'AP-species01': 39.793645945563384,
               'AP-sphaeridia': 37.81927923871891})])

In [21]:
print("Stas ist ein Noobmaster.")

Stas ist ein Noobmaster.
